<a href="https://colab.research.google.com/github/Yuant9/visual-recognition-MNIST-double-digit-project-/blob/master/mnistdd_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from skimage.draw import polygon
import numpy as np
from PIL import Image, ImageDraw
import tensorflow as tf
from tensorflow import layers as tl
import os

In [0]:
!pip install -U matplotlib
def mkdirs(dirs):
  
    for d in dirs:
        if not os.path.exists(d):
            os.mkdir(d)
        elif not os.path.isdir(d):
            os.remove(d)
            os.mkdir(d)
train = False
mkdirs(["model","model/detect", "model/classify", "model/combine", "sample", "sample/detect", "sample/classify", "sample/combine", "log", "log/detect", "log/classify", "log/combine"])

Requirement already up-to-date: matplotlib in /usr/local/lib/python3.6/dist-packages (3.0.2)


In [0]:
def compute_classification_acc(pred, gt):
    # pred and gt are both
    assert pred.shape == gt.shape
    return (pred == gt).astype(int).sum() / gt.size


def compute_iou(b_pred, b_gt):
    # b_pred: predicted bounding boxes, shape=(n,2,4)
    # b_gt: ground truth bounding boxes, shape=(n,2,4)

    n = np.shape(b_gt)[0]
    L_pred = np.zeros((64, 64))
    L_gt = np.zeros((64, 64))
    iou = 0.0
    for i in range(n):
        for b in range(2):
            rr, cc = polygon([b_pred[i, b, 0], b_pred[i, b, 0], b_pred[i, b, 2], b_pred[i, b, 2]],
                             [b_pred[i, b, 1], b_pred[i, b, 3], b_pred[i, b, 3], b_pred[i, b, 1]], [64, 64])
            L_pred[rr, cc] = 1

            rr, cc = polygon([b_gt[i, b, 0], b_gt[i, b, 0], b_gt[i, b, 2], b_gt[i, b, 2]],
                             [b_gt[i, b, 1], b_gt[i, b, 3], b_gt[i, b, 3], b_gt[i, b, 1]], [64, 64])
            L_gt[rr, cc] = 1

            iou += (1.0 / (2 * n)) * (np.sum((L_pred + L_gt) == 2) / np.sum((L_pred + L_gt) >= 1))

            L_pred[:, :] = 0
            L_gt[:, :] = 0

    return iou


def evaluation(pred_class, pred_bboxes, prefix="valid"):
    # pred_class: Your predicted labels for the 2 digits, shape [N, 2]
    # pred_bboxes: Your predicted bboxes for 2 digits, shape [N, 2, 4]
    gt_class = np.load(prefix + "_Y.npy")
    gt_bboxes = np.load(prefix + "_bboxes.npy")
    acc = compute_classification_acc(pred_class, gt_class)
    iou = compute_iou(pred_bboxes, gt_bboxes)
    print(f"Classification Acc: {acc}")
    print(f"BBoxes IOU: {iou}")

In [0]:
def part_iou(pred, gt):
    L_pred = np.zeros((64, 64))
    L_gt = np.zeros((64, 64))
    rr, cc = polygon([pred[0], pred[0], pred[2], pred[2]],
                     [pred[1], pred[3], pred[3], pred[1]], [64, 64])
    L_pred[rr, cc] = 1
    rr, cc = polygon([gt[0], gt[0], gt[2], gt[2]],
                     [gt[1], gt[3], gt[3], gt[1]], [64, 64])
    L_gt[rr, cc] = 1
    return np.sum((L_pred + L_gt) == 2) / np.sum((L_pred + L_gt) >= 1)

def adjust_predict_bboxes(predict_bboxes, bboxes):  # Adjust order according to IOU
    assert len(predict_bboxes) == len(bboxes)
    n = len(predict_bboxes)
    result = np.zeros_like(predict_bboxes)
    for i in range(n):
        t1 = part_iou(predict_bboxes[i, 0], bboxes[i, 0]) + part_iou(predict_bboxes[i, 0], bboxes[i, 0])
        t2 = part_iou(predict_bboxes[i, 0], bboxes[i, 1]) + part_iou(predict_bboxes[i, 1], bboxes[i, 0])
        if t1 < t2:
            result[i] = predict_bboxes[i, ::-1]
        else:
            result[i] = predict_bboxes[i]
    return result




def save_model(ckpt, out):  # Save the model to .npz file
    reader = tf.pywrap_tensorflow.NewCheckpointReader(ckpt)
    var_to_shape_map = reader.get_variable_to_shape_map()
    model = {}
    for key in var_to_shape_map.keys():
        if "Adam" not in key and ("kernel" in key or "bias" in key):
            model[key] = reader.get_tensor(key)
    np.savez(out, **model)


def crop_image(input_x, input_y, input_bboxes):  # Crop image
    assert len(input_x) == len(input_y) and len(input_x) == len(input_bboxes)
    size = len(input_x)
    output_x = np.zeros([size * 2, 28 * 28], np.uint8)
    output_y = input_y.reshape([size * 2]).astype(np.int64)
    for i in range(size):
        for j in range(2):
            output_x[i * 2 + j, :] = input_x[i].reshape((64, 64))[input_bboxes[i, j, 0]:input_bboxes[i, j, 2], input_bboxes[i, j, 1]:input_bboxes[i, j, 3]].flatten()
    return output_x, output_y


def crop_image_expand_rand_count(input_x, input_y, input_bboxes, pad=4, count=10):  # Random offset cropping image
    assert len(input_x) == len(input_y) and len(input_x) == len(input_bboxes)
    size = len(input_x)
    output_x = np.zeros([size * 2 * count, 28 * 28], np.uint8)
    output_y = np.zeros(size * 2 * count, np.int64)
    base = np.zeros((64 + pad * 2, 64 + pad * 2), np.uint8)
    for i in range(size):
        for j in range(2):
            base[pad:pad + 64, pad:pad + 64] = input_x[i].reshape((64, 64))
            b = np.tile(np.array([np.random.randint(-pad, pad, count), np.random.randint(-pad, pad, count)]).transpose(), 2) + input_bboxes[i, j] + [pad, pad, pad, pad]
            for k in range(count):
                output_x[(i * 2 + j) * count + k, :] = base[b[k, 0]:b[k, 2], b[k, 1]:b[k, 3]].flatten()
                output_y[(i * 2 + j) * count + k] = input_y[i, j]
    return output_x, output_y

In [0]:
def model_classify(pre_model=None, reuse=None):  # Classification model that supports loading parameters from npz files
    with tf.variable_scope("classify", reuse=reuse):
        x = tf.placeholder(tf.uint8, [None, 28 * 28])
        y = tf.placeholder(tf.int64, [None])
        h_00 = tf.divide(tf.cast(tf.reshape(x, [-1, 28, 28, 1]), tf.float32), 255.)  # 28x28x1
        if pre_model is not None:
            h_01 = tl.conv2d(h_00, 16, 5, padding="same", activation=tf.nn.leaky_relu, kernel_initializer=tf.initializers.constant(pre_model["classify/conv2d_1/kernel"]), bias_initializer=tf.initializers.constant(pre_model["classify/conv2d_1/bias"]), name="conv2d_1")  # 28x28x16
            h_02 = tl.max_pooling2d(h_01, 2, 2)  # 14x14x16
            h_03 = tl.conv2d(h_02, 64, 5, padding="same", activation=tf.nn.leaky_relu, kernel_initializer=tf.initializers.constant(pre_model["classify/conv2d_2/kernel"]), bias_initializer=tf.initializers.constant(pre_model["classify/conv2d_2/bias"]), name="conv2d_2")  # 14x14x64
            h_04 = tl.max_pooling2d(h_03, 2, 2)  # 7x7x64
            h_05 = tl.flatten(h_04)
            h_06 = tl.dense(h_05, 1024, activation=tf.nn.leaky_relu, kernel_initializer=tf.initializers.constant(pre_model["classify/dense_3/kernel"]), bias_initializer=tf.initializers.constant(pre_model["classify/dense_3/bias"]), name="dense_3")
            dropout_prob = tf.placeholder(tf.float32)
            h_07 = tl.dropout(h_06, dropout_prob)
            h_08 = tl.dense(h_07, 10, kernel_initializer=tf.initializers.constant(pre_model["classify/dense_4/kernel"]), bias_initializer=tf.initializers.constant(pre_model["classify/dense_4/bias"]), name="dense_4")
        else:
            h_01 = tl.conv2d(h_00, 16, 5, padding="same", activation=tf.nn.leaky_relu, name="conv2d_1")  # 28x28x16
            h_02 = tl.max_pooling2d(h_01, 2, 2)  # 14x14x16
            h_03 = tl.conv2d(h_02, 64, 5, padding="same", activation=tf.nn.leaky_relu, name="conv2d_2")  # 14x14x64
            h_04 = tl.max_pooling2d(h_03, 2, 2)  # 7x7x64
            h_05 = tl.flatten(h_04)
            h_06 = tl.dense(h_05, 1024, activation=tf.nn.leaky_relu, name="dense_3")
            dropout_prob = tf.placeholder(tf.float32)
            h_07 = tl.dropout(h_06, dropout_prob)
            h_08 = tl.dense(h_07, 10, name="dense_4")
        predict_label = tf.argmax(h_08, axis=1)
        loss_classify = tf.losses.sparse_softmax_cross_entropy(y, h_08)
        classify_accuracy = tf.reduce_mean(tf.cast(tf.equal(predict_label, y), tf.float32))
    return x, y, dropout_prob, predict_label, loss_classify, classify_accuracy

In [0]:
def train_classify(start_step=0, restore=False):  # Train the classification model and save the model as an npz file
    train_x = np.load("train_X.npy")
    train_y = np.load("train_Y.npy")
    train_bboxes = np.load("train_bboxes.npy")
    valid_x = np.load("valid_X.npy")
    valid_y = np.load("valid_Y.npy")
    valid_bboxes = np.load("valid_bboxes.npy")

    train_x, train_y = crop_image_expand_rand_count(train_x, train_y, train_bboxes, 4, 10)
    valid_x, valid_y = crop_image(valid_x, valid_y, valid_bboxes)

    m_x, m_y, m_dropout_prob, m_predict_label, m_loss_classify, m_classify_accuracy = model_classify()

    tf.summary.scalar("loss_classify", m_loss_classify)
    merged_summary_op = tf.summary.merge_all()

    classify_global_step = tf.Variable(0, trainable=False)
    classify_lr = tf.train.exponential_decay(learning_rate=2e-6, global_step=classify_global_step, decay_steps=2000, decay_rate=0.99)
    classify_op = tf.train.AdamOptimizer(classify_lr).minimize(m_loss_classify)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=10)

    if restore:
        saver.restore(sess, "model/classify/model.ckpt-%d" % start_step)
    summary_writer = tf.summary.FileWriter("log/classify", sess.graph)

    for step in range(start_step, 100001):
        sample = np.random.randint(0, len(train_x), 100)
        if step % 100 == 0:
            if step % 1000 == 0:
                v_predict_label, v_classify_accuracy = sess.run([m_predict_label, m_classify_accuracy], feed_dict={m_x: valid_x, m_y: valid_y})
                print("step = %5d, classify_accuracy = %f" % (step, v_classify_accuracy))
                image = np.ones((100 * 28 + 101, 100 * 28 + 101), np.uint8) * 255
                for i in range(100):
                    for j in range(100):
                        k = i * 100 + j
                        pl = v_predict_label[k]
                        img = Image.fromarray(valid_x[k].reshape([28, 28]))
                        img = generate_image(img, label=[pl])
                        image[i * 29 + 1:i * 29 + 29, j * 29 + 1:j * 29 + 29] = np.array(img)
                Image.fromarray(image).save("sample/classify/%05d.jpg" % step)
                saver.save(sess, "model/classify/model.ckpt", global_step=step)
            summary, v_classify_lr, v_loss_classify = sess.run([merged_summary_op, classify_lr, m_loss_classify], feed_dict={m_x: train_x[sample], m_y: train_y[sample], classify_global_step: step})
            print("step = %5d, lr = %g, loss_classify = %f" % (step, v_classify_lr, v_loss_classify))
            summary_writer.add_summary(summary, step)
        sess.run(classify_op, feed_dict={m_x: train_x[sample], m_y: train_y[sample], m_dropout_prob: 0.5, classify_global_step: step})
    save_model("model/classify/model.ckpt-%d" % 100000, "classify.npz")
    sess.close()

In [0]:
if train:
    with tf.variable_scope("train_classify", reuse=tf.AUTO_REUSE):
        train_classify(0, False)  # Before running the next training, you need to restart the service, execute the previous function, skip this train

In [0]:
def model_detect(pre_model=None, reuse=None):  # Target detection model, applying migration learning, loading some parameters from the classification model
    with tf.variable_scope("detect", reuse=reuse):
        x = tf.placeholder(tf.uint8, [None, 64 * 64])
        bboxes = tf.placeholder(tf.uint8, [None, 2, 4])
        center = tf.add(*tf.split(bboxes, 2, 2)) / 2 - 32
        h_00 = tf.divide(tf.cast(tf.reshape(x, [-1, 64, 64, 1]), tf.float32), 255.)  # 64x64x1
        if pre_model is not None:
            h_01 = tl.conv2d(h_00, 16, 5, padding="same", activation=tf.nn.leaky_relu, kernel_initializer=tf.initializers.constant(pre_model["detect/conv2d_1/kernel"]), bias_initializer=tf.initializers.constant(pre_model["detect/conv2d_1/bias"]), name="conv2d_1")  # 64x64x16
            h_02 = tl.max_pooling2d(h_01, 2, 2)  # 32x32x16
            h_03 = tl.conv2d(h_02, 64, 5, padding="same", activation=tf.nn.leaky_relu, kernel_initializer=tf.initializers.constant(pre_model["detect/conv2d_2/kernel"]), bias_initializer=tf.initializers.constant(pre_model["detect/conv2d_2/bias"]), name="conv2d_2")  # 32x32x64
            h_04 = tl.max_pooling2d(h_03, 2, 2)  # 16x16x64
            h_05 = tl.flatten(h_04)
            h_06 = tl.dense(h_05, 4096, activation=tf.nn.tanh, kernel_initializer=tf.initializers.constant(pre_model["detect/dense_3/kernel"]), bias_initializer=tf.initializers.constant(pre_model["detect/dense_3/bias"]), name="dense_3")
            dropout_prob = tf.placeholder(tf.float32)
            h_07 = tl.dropout(h_06, dropout_prob)
            h_08 = tl.dense(h_07, 4, kernel_initializer=tf.initializers.constant(pre_model["detect/dense_4/kernel"]), bias_initializer=tf.initializers.constant(pre_model["detect/dense_4/bias"]), name="dense_4")
        else:
            pre_model = np.load("classify.npz")
            h_01 = tl.conv2d(h_00, 16, 5, padding="same", activation=tf.nn.leaky_relu, kernel_initializer=tf.initializers.constant(pre_model["classify/conv2d_1/kernel"]), bias_initializer=tf.initializers.constant(pre_model["classify/conv2d_1/bias"]), name="conv2d_1")  # 64x64x16
            h_02 = tl.max_pooling2d(h_01, 2, 2)  # 32x32x16
            h_03 = tl.conv2d(h_02, 64, 5, padding="same", activation=tf.nn.leaky_relu, kernel_initializer=tf.initializers.constant(pre_model["classify/conv2d_2/kernel"]), bias_initializer=tf.initializers.constant(pre_model["classify/conv2d_2/bias"]), name="conv2d_2")  # 32x32x64
            h_04 = tl.max_pooling2d(h_03, 2, 2)  # 16x16x64
            h_05 = tl.flatten(h_04)
            h_06 = tl.dense(h_05, 4096, activation=tf.nn.tanh, name="dense_3")
            dropout_prob = tf.placeholder(tf.float32)
            h_07 = tl.dropout(h_06, dropout_prob)
            h_08 = tl.dense(h_07, 4, name="dense_4")
        predict_center = tf.clip_by_value(tf.reshape(h_08, [-1, 2, 2]), -18, 18)
        predict_center_round = tf.round(predict_center)
        overlap_h, overlap_v = tf.split(tf.maximum(28 - tf.abs(predict_center_round - center), 0), 2, 2)
        overlap = overlap_h * overlap_v
        iou = tf.reduce_mean(overlap / (2 * 28 * 28 - overlap))
        predict_bboxes = tf.concat([predict_center_round + 32 - 14, predict_center_round + 32 + 14], 2)
        loss_detect = tf.losses.absolute_difference(center, predict_center)
    return x, bboxes, dropout_prob, predict_bboxes, loss_detect, iou

In [0]:
def train_detect(start_step=0, restore=False):  # Training target detection model
    train_x = np.load("train_X.npy")
    train_bboxes = np.load("train_bboxes.npy")
    valid_x = np.load("valid_X.npy")
    valid_bboxes = np.load("valid_bboxes.npy")

    m_x, m_bboxes, m_dropout_prob, m_predict_bboxes, m_loss_detect, m_iou = model_detect()

    tf.summary.scalar("loss_detect", m_loss_detect)
    merged_summary_op = tf.summary.merge_all()

    detect_global_step = tf.Variable(0, trainable=False)
    detect_lr = tf.train.exponential_decay(learning_rate=0.001, global_step=detect_global_step, decay_steps=100, decay_rate=0.995)
    detect_op = tf.train.AdamOptimizer(detect_lr).minimize(m_loss_detect)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=10)

    if restore:
        saver.restore(sess, "model/detect/model.ckpt-%d" % start_step)
    summary_writer = tf.summary.FileWriter("log/detect", sess.graph)

    for step in range(start_step, 50001):
        sample = np.random.randint(0, len(train_x), 100)
        if step % 100 == 0:
            if step % 1000 == 0:
                v_predict_bboxes, v_iou = sess.run([m_predict_bboxes, m_iou], feed_dict={m_x: valid_x, m_bboxes: valid_bboxes})
                iou = compute_iou(valid_bboxes, v_predict_bboxes)
                print("step = %5d, detect_iou = %f,  iou = %f" % (step, iou, v_iou))
                image = np.ones((100 * 64 + 101, 50 * 64 + 51), np.uint8) * 255
                for i in range(100):
                    for j in range(50):
                        k = i * 50 + j
                        pb = v_predict_bboxes[k]
                        img = Image.fromarray(valid_x[k].reshape([64, 64]))
                        img = generate_image(img, bbox=pb)
                        image[i * 65 + 1:i * 65 + 65, j * 65 + 1:j * 65 + 65] = np.array(img)
                Image.fromarray(image).save("sample/detect/%05d.jpg" % step)
                saver.save(sess, "model/detect/model.ckpt", global_step=step)
            summary, v_detect_lr, v_loss_detect, v_predict_bboxes, v_iou = sess.run([merged_summary_op, detect_lr, m_loss_detect, m_predict_bboxes, m_iou], feed_dict={m_x: train_x[sample], m_bboxes: train_bboxes[sample], detect_global_step: step})
            print("step = %5d, lr = %g, loss_detect = %f, iou = %f" % (step, v_detect_lr, v_loss_detect, v_iou))
            summary_writer.add_summary(summary, step)
        sess.run(detect_op, feed_dict={m_x: train_x[sample], m_bboxes: train_bboxes[sample], m_dropout_prob: 0.5, detect_global_step: step})
    save_model("model/detect/model.ckpt-%d" % 50000, "detect.npz")
    sess.close()

In [0]:
if train:
    with tf.variable_scope("train_detect", reuse=tf.AUTO_REUSE):
        train_detect(0, False) 

In [0]:
def train_combine(start_step=0, restore=False):  # Joint training, fine-tuning classification model
    train_x = np.load("train_X.npy")
    train_y = np.load("train_Y.npy")
    train_bboxes = np.load("train_bboxes.npy")
    valid_x = np.load("valid_X.npy")
    valid_y = np.load("valid_Y.npy")
    valid_bboxes = np.load("valid_bboxes.npy")

    md_x, _, _, md_predict_bboxes, _, m_iou = model_detect(np.load("model/detect.npz"), tf.AUTO_REUSE)
    mc_x, mc_y, mc_dropout_prob, mc_predict_label, mc_loss_classify, mc_classify_accuracy = model_classify(np.load("model/classify.npz"), tf.AUTO_REUSE)

    tf.summary.scalar("loss_classify", mc_loss_classify)
    merged_summary_op = tf.summary.merge_all()

    classify_global_step = tf.Variable(0, trainable=False)
    classify_lr = tf.train.exponential_decay(learning_rate=1e-6, global_step=classify_global_step, decay_steps=1000, decay_rate=0.99)
    classify_op = tf.train.AdamOptimizer(classify_lr).minimize(mc_loss_classify)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=10)
    if restore:
        saver.restore(sess, "model/combine/model.ckpt-%d" % start_step)

    summary_writer = tf.summary.FileWriter("log", sess.graph)

    for step in range(start_step, 5001):
        sample = np.random.randint(0, len(train_x), 100)
        vd_predict_bboxes = sess.run(md_predict_bboxes, feed_dict={md_x: train_x[sample]})
        vd_predict_bboxes = adjust_predict_bboxes(np.round(vd_predict_bboxes).astype(np.uint8), train_bboxes[sample])
        sample_x, sample_y = crop_image(train_x[sample], train_y[sample], vd_predict_bboxes)
        if step % 100 == 0:
            if step % 1000 == 0:
                vd_predict_bboxes = sess.run(md_predict_bboxes, feed_dict={md_x: valid_x})
                vd_predict_bboxes = adjust_predict_bboxes(np.round(vd_predict_bboxes).astype(np.uint8), valid_bboxes)
                sample_x, sample_y = crop_image(valid_x, valid_y, vd_predict_bboxes)
                vc_predict_label, vc_classify_accuracy = sess.run([mc_predict_label, mc_classify_accuracy], feed_dict={mc_x: sample_x, mc_y: sample_y})
                vc_predict_label = vc_predict_label.reshape([5000, 2])
                image = np.ones((100 * 64 + 101, 50 * 64 + 51), np.uint8) * 255
                for i in range(100):
                    for j in range(50):
                        k = i * 50 + j
                        pl = vc_predict_label[k]
                        pb = vd_predict_bboxes[k]
                        img = Image.fromarray(valid_x[k].reshape([64, 64]))
                        img = generate_image(img, label=pl, bbox=pb)
                        image[i * 65 + 1:i * 65 + 65, j * 65 + 1:j * 65 + 65] = np.array(img)
                Image.fromarray(image).save("sample/combine/%05d.jpg" % step)
                evaluation(vc_predict_label, vd_predict_bboxes, "train_val/valid")
                saver.save(sess, "model/combine/model.ckpt", global_step=step)
            summary, v_classify_lr, v_loss_classify = sess.run([merged_summary_op, classify_lr, mc_loss_classify], feed_dict={mc_x: sample_x, mc_y: sample_y, classify_global_step: step})
            print("step = %5d, lr = %g, loss_classify = %f" % (step, v_classify_lr, v_loss_classify))
            summary_writer.add_summary(summary, step)
        sess.run(classify_op, feed_dict={mc_x: sample_x, mc_y: sample_y, mc_dropout_prob: 0.5, classify_global_step: step})
    save_model("model/combine/model.ckpt-%d" % 5000, "combine.npz")
    sess.close()

In [0]:
if train:
    with tf.variable_scope("train_combine", reuse=tf.AUTO_REUSE):
        train_combine(0, False)  # Before running the next testing, you need to restart the service, execute the previous function, skip this train

In [0]:
import matplotlib.pyplot as plt
def test_combine():  # Test and output
    valid_x = np.load("valid_X.npy")#If you are running the test, please change the file name
    
    model_data = np.load("combine.npz")
    md_x, _, _, md_predict_bboxes, _, m_iou = model_detect(model_data, tf.AUTO_REUSE)
    mc_x, mc_y, mc_dropout_prob, mc_predict_label, mc_loss_classify, mc_classify_accuracy = model_classify(model_data, tf.AUTO_REUSE)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    vd_predict_bboxes = sess.run(md_predict_bboxes, feed_dict={md_x: valid_x})
    sample_x, _ = crop_image(valid_x, np.zeros((len(valid_x), 2), np.uint8), vd_predict_bboxes.astype(np.uint8))
    vc_predict_label = sess.run(mc_predict_label, feed_dict={mc_x: sample_x})
    vc_predict_label = vc_predict_label.reshape([5000, 2])  #if you are testing, please chage the argument [5000,2] to [1000,2]
   
    # if you are runing a test,please change the third arguement "valid" to "test"
    evaluation(vc_predict_label, vd_predict_bboxes, "valid")

In [0]:
with tf.variable_scope("test_combine", reuse=tf.AUTO_REUSE):
    test_combine()  #to run the evaluation function provided on eclass, just run this function since evaluation is called in this function

Classification Acc: 0.9724
BBoxes IOU: 0.998981012919552
